# Reproduce the baseline
This notebook provides the baseline results on `l2rpn_case14_sandbox` environment obtained through various models (aka augmented simulators) available in LIPS framework. It starts by importing the required dataset, instantiating the benchmark, training an existing augmented simulator and finally it shows how the evaluation is performed to reproduce the baseline performance.

NB. The reference dataset for the competition is not `l2rpn_case14_sandbox` (14 substations) but `l2rpn_idf_2023` (118 substations). We use in this notebook a smaller power grid for faster computations and illustrative purposes.

### Prerequisites

Install the LIPS framework if it is not already done. For more information look at the LIPS framework [Github repository](https://github.com/IRT-SystemX/LIPS) 

#### For developments on local machine

In [ ]:
### Install a virtual environment
# Option 1:  using conda (recommended)
!conda create -n venv_lips python=3.10
!conda activate venv_lips

# Option 2: using virtualenv
!pip install virtualenv
!virtualenv -p /usr/bin/python3.10 venv_lips
!source venv_lips/bin/activate

### Install the LIPS framework
# Option 1: Get the last version of LIPS framework from PyPI (Recommended)
!pip install lips-benchmark .[recommended]

# Option 2: Get the last version from github repository
!git clone https://github.com/IRT-SystemX/LIPS.git
!pip install -U LIPS/.[recommended]

#### For Google Colab Users
You could also use a GPU device from `Runtime > Change runtime type` and by selecting `T4 GPU`.

In [ ]:
### Install the LIPS framework
# Option 1: Get the last version of LIPS framework from PyPI (Recommended)
!pip install lips-benchmark .[recommended]

In [ ]:
# Option 2: Get the last version from github repository
!git clone https://github.com/IRT-SystemX/LIPS.git
!pip install -U LIPS/.[recommended]

Attention: You may restart the session after this installation, in order that the changes be effective.

In [ ]:
# Clone the starting kit
!git clone https://github.com/IRT-SystemX/ml4physim_startingkit_powergrid.git
# and change the directory to the starting kit to be able to run correctly this notebook
import os
os.chdir("ml4physim_startingkit_powergrid")

### Import the dataset

In [ ]:
### Import required packages
import os
from lips.benchmark.powergridBenchmark import PowerGridBenchmark

Define the required paths

In [ ]:
BENCH_CONFIG_PATH = os.path.join("configs", "benchmarks", "lips_case14_sandbox.ini")
DATA_PATH = os.path.join("input_data_local", "lips_case14_sandbox")
TRAINED_MODELS = os.path.join("input_data_local", "trained_models")
LOG_PATH = "logs.log"

Download the dataset

The already provided datasets on starting kit are demo versions of the complet datasets. The complet datasets should be downloaded using the following function and replace the demo versions.

**NB.** <span style="color: red">The challenge dataset is based on `lips_idf_2023` environment and all the solutions should be trained and evaluated on this dataset.</span> This notebook illustrates the procedure of reproducing the baseline results for a smaller environment with only 14 nodes. This could be used for new users to learn a little bit more about power grids and flow dynamics.

In [ ]:
## Download the dataset through the dedicated lips function
from lips.dataset.powergridDataSet import downloadPowergridDataset

downloadPowergridDataset("input_data_local", "lips_case14_sandbox")

## Benchmark
### First step: load the dataset

In [ ]:
benchmark_kwargs = {"attr_x": ("prod_p", "prod_v", "load_p", "load_q"),
                    "attr_y": ("a_or", "a_ex", "p_or", "p_ex", "v_or", "v_ex"),
                    "attr_tau": ("line_status", "topo_vect"),
                    "attr_physics": None}

benchmark = PowerGridBenchmark(benchmark_path=DATA_PATH,
                               config_path=BENCH_CONFIG_PATH,
                               benchmark_name="Benchmark_competition",
                               load_data_set=True, 
                               load_ybus_as_sparse=False,
                               log_path=LOG_PATH,
                               **benchmark_kwargs)

In [ ]:
benchmark.train_dataset.data.keys()

Once the benchmark is instantiated, we can verify the corresponding configurations imported from the configuration file indicated in `BENCH_CONFIG_PATH` and `benchmark_name` section.

In [ ]:
benchmark.config.get_options_dict()

### Second step: Select a model (aka Augmented Simulator)

Here we present the tensorflow based augmented simulators for learning a physical domain. Two different architectures are included for the moment in LIPS framework, which are : 
- Fully Connected Neural Network
- LeapNet Neural network

The tensorflow side implementations are a little bit different from torch based implementations. In order to generalize on more architectures, we allow that each model (architecture) be a subclass of `TensorflowSimulator` base class. The main functions to `train`, `evaluate`, `load`, `save` models are implemented in base class and could be used directly by sub-classes without any overloading. Some specific tasks as data preparation and post processing of predictions could be done using the child classes.

Select the GPU device if there is one (<span style="color:red">Dont run this cell in Google Colab</span>).

In [ ]:
import tensorflow as tf

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

memory_limit = 20000

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memory_limit)])
  except RuntimeError as e:
    print(e)

We start by importing the required architecture and optionally a scaler used to normalize the dataset.

In [ ]:
from lips.augmented_simulators.tensorflow_models import TfFullyConnected
from lips.dataset.scaler import StandardScaler

In this example, we select the fully connected architecture with its corresponding configuration file.

In [ ]:
# Indicate the path required for corresponding augmented simulator parameters
SIM_CONFIG_PATH = os.path.join("configs", "simulators", "tf_fc.ini")

In [ ]:
tf_fc = TfFullyConnected(name="tf_fc",
                         bench_config_path=BENCH_CONFIG_PATH,
                         bench_config_name="Benchmark_competition",
                         bench_kwargs=benchmark_kwargs,
                         sim_config_path=SIM_CONFIG_PATH,
                         sim_config_name="DEFAULT",
                         scaler=StandardScaler,
                         log_path=LOG_PATH)

We can print the hyperparameters of the selected architecture by using the `params` attribute. These hyperparameters corresponds to the configuration file indicated by `SIM_CONFIG_PATH` and `sim_config_name` section.

In [ ]:
tf_fc.params

### Third step: Train the augmented simulator

Verify the inputs and outputs of the model

In [ ]:
# the process_dataset function is called inside the train function call to prepare the data for training
inputs, outputs = tf_fc.process_dataset(benchmark.train_dataset, training=True)

In [ ]:
output_dim = 0
input_dim = 0
for var_name in benchmark_kwargs["attr_y"]:
    output_dim += benchmark.train_dataset.data.get(var_name).shape[1]

for var_name in (benchmark_kwargs["attr_x"] + benchmark_kwargs["attr_tau"]):
    input_dim += benchmark.train_dataset.data.get(var_name).shape[1]

print("input_dim: ", input_dim)
print("output_dim: ", output_dim)

assert(inputs.shape[1] == input_dim)
assert(outputs.shape[1] == output_dim)

Train your model

In [ ]:
tf_fc.train(train_dataset=benchmark.train_dataset,
            val_dataset=benchmark.val_dataset,
            epochs=2
           )

You can also save and load the model fitted parameters alongside its meta data using the following functions.

Save your model

In [ ]:
SAVE_PATH = os.path.join(TRAINED_MODELS, benchmark.env_name)
tf_fc.save(SAVE_PATH)

Load your trained model

In [ ]:
from lips.augmented_simulators.tensorflow_models import TfFullyConnected
from lips.dataset.scaler import StandardScaler

# Indicate the path required for corresponding augmented simulator parameters
SIM_CONFIG_PATH = os.path.join("configs", "simulators", "tf_fc.ini")

tf_fc = TfFullyConnected(name="tf_fc",
                         bench_config_path=BENCH_CONFIG_PATH,
                         bench_config_name="Benchmark_competition",
                         bench_kwargs=benchmark_kwargs,
                         sim_config_path=SIM_CONFIG_PATH,
                         sim_config_name="DEFAULT",
                         scaler=StandardScaler,
                         log_path=LOG_PATH)

LOAD_PATH = os.path.join(TRAINED_MODELS, "fully_connected")
tf_fc.restore(path=LOAD_PATH)

You can visualize the convergence of the model using `visualize_convergence` of the augmented simulator object.

In [ ]:
tf_fc.visualize_convergence()

Summary of the model (layers and shapes)

In [ ]:
tf_fc.summary()

### Fourth step: Evaluate the augmented simulator
In this section, we use the evaluation module of LIPS framework to evaluate the trained augmented simulator. We can see which evaluaton criteria are used to evaluate the performance of the model:

In [ ]:
from pprint import pprint
pprint(benchmark.config.get_option("eval_dict"))

To evaluate the augmented simulator, we call simply the `evaluate_simulator` function of the benchmark class, which will be instantiate an evaluation object from the corresponding `PowerGridEvaluation` class and intitialize it with the benchmark configuration file. This function get various arguments:
- `augmented_simulator`: Which is the trained augmented simulator;
- `eval_batch_size`: the batch size used during the evaluation of the augmented simulator;
- `dataset`: a string indicating on which dataset, the evaluation should be performed. The options are `all` for three datasets, `val` for validation dataset only, `test` for test dataset only and `test_ood_topo` for out-of-distribution dataset only.
- `shuffle`: whether to shuffle the dataset for the evaluation.
- `save_path` and `save_predictions`: parameters allowing to save the evaluation results in indicated path and save the predictions of the model.

In [ ]:
# EVAL_SAVE_PATH = get_path(EVALUATION_PATH, benchmark1)
tf_fc_metrics = benchmark.evaluate_simulator(augmented_simulator=tf_fc,
                                             eval_batch_size=128,
                                             dataset="all",
                                             shuffle=False,
                                             save_path=None,
                                             save_predictions=False
                                            )

In [ ]:
tf_fc_metrics.keys()

In [ ]:
tf_fc_metrics["test"].keys()

In [ ]:
tf_fc_metrics["test"]["ML"]

In [ ]:
tf_fc_metrics["test"]["Physics"]

In [ ]:
tf_fc_metrics["test_ood_topo"]["ML"]

## LeapNet Architecture

In comparison to Fully Connected architecture, used in the previous section, where the topology vector (`topo_vect`) and power lines connectivity vector (`line_status`) are used directly as the inputs of the architecture, in LeapNet architecture (see [corresponding article](https://www.sciencedirect.com/science/article/abs/pii/S0925231220305051)) considers the topology vector in latent dimension to take into account the various topology configurations as shown in figure below.

![image.png](img/leap_net.png)

Herein, we get the list of reference topology actions which will be given to LeapNet architecture for their encoding.

In [ ]:
topo_actions = benchmark.config.get_option("dataset_create_params")["reference_args"]["topo_actions"]

kwargs_tau = []
for el in topo_actions:
     kwargs_tau.append(el["set_bus"]["substations_id"][0])

In [ ]:
# Indicate the path required for corresponding augmented simulator parameters
SIM_CONFIG_PATH = os.path.join("configs", "simulators", "tf_leapnet.ini")

In [ ]:
import tensorflow as tf
from lips.augmented_simulators.tensorflow_models.powergrid.leap_net import LeapNet
from lips.dataset.scaler.powergrid_scaler import PowerGridScaler

leap_net = LeapNet(name="tf_leapnet",                  
                   bench_config_path=BENCH_CONFIG_PATH,
                   bench_config_name="Benchmark_competition",
                   bench_kwargs=benchmark_kwargs,
                   sim_config_path=SIM_CONFIG_PATH,
                   sim_config_name="DEFAULT", 
                   log_path=LOG_PATH,
                   loss = {"name": "mse"},
                   lr = 1e-4,
                   activation = tf.keras.layers.LeakyReLU(alpha=0.01),
                   sizes_enc=(),
                   sizes_main=(200, 200),
                   sizes_out=(),
                   topo_vect_to_tau="given_list",
                   kwargs_tau = kwargs_tau,
                   layer = "resnet",
                   scale_main_layer = 200,
                   scale_input_dec_layer = 200,
                   mult_by_zero_lines_pred = False,
                   scaler = PowerGridScaler,
                  )

In [ ]:
leap_net.train(train_dataset=benchmark.train_dataset,
               val_dataset=benchmark.val_dataset,
               batch_size=256,
               epochs=400)

In [ ]:
SAVE_PATH = os.path.join(TRAINED_MODELS, benchmark.env_name)
leap_net.save(SAVE_PATH)

In [ ]:
leap_net.visualize_convergence()

In [ ]:
# EVAL_SAVE_PATH = get_path(EVALUATION_PATH, benchmark1)
leapnet_metrics = benchmark.evaluate_simulator(augmented_simulator=leap_net,
                                               eval_batch_size=100000,
                                               dataset="all",
                                               shuffle=False,
                                               save_path=None,
                                               save_predictions=False
                                              )

In [ ]:
leapnet_metrics["test"]["ML"]

In [ ]:
leapnet_metrics["test"]["Physics"]

In [ ]:
leapnet_metrics["test_ood_topo"]["ML"]

In [ ]:
leapnet_metrics["test_ood_topo"]["Physics"]